## Homework 4 for ECON-GA 3002, Section 12.

**{Name, N-Number, Email}**: {Arnav Sood, N11193569, asood@nyu.edu}

**Overview**: Show the downward bias in the standard OLS estimate of the lag coefficient by running a large number of trials for a few (alpha, n) pairs, and then plotting. Optimize for speed.

**Disclosure**: Was not written using vim. Was instead written using a Jupyter notebook and REPL,across Linux and OS X.

> Vanilla Python version. Before optimizing, get a vanilla code set in Python.

**Update: Parallelization is Hard**

**The optimization path will probably look something like this:**
    - Write vanilla Python code.
            - Wall Time: 2m41s
    - Make easy Python improvements (e.g, @jit, numbafy)
            - @jit: Wall Time: 17.9s
            - @jit on Mac (no VM): 11.3s
            - add numpy: 2s?
            - remove print: like 5s?
    - Make harder Python improvements (not sure what they are a priori)
    - Write code in C if it makes sense
    - Figure out what PEP8 is, and make sure I'm using it

> Perform some setup tasks.

In [9]:
# Throw in numba jit compile.
from numba import jit

# Add more numpy stuff.
import numpy as np

# Setup variables.
alphas = np.arange(0.5,1,0.1)
ns = np.arange(50,550,50)

> Write a function to take an (alpha, n, beta) tuple, and return the bias for the estimate.
> Write also a function to take all the values, and compute an estimate

In [10]:
estimate = lambda sumx, sumy, sumxsq, sumxy, n : (sumxy - (n-1)*(sumx/(n-1))*(sumy/(n-1)))/(sumxsq - (n-1)*math.pow((sumx/(n-1)),2));

@jit
def bias(alpha, n, beta):
        sumx = 0
        sumy = 0
        sumxy = 0
        sumxsq = 0
        x = rand()*100
        for i in range(n):
            sumx += x
            sumxsq += math.pow(x,2)
            y = beta + alpha*x + rand()
            sumxy += x*y
            sumy += y
            x = y
        val = estimate(sumx, sumy, sumxsq, sumxy, n) - alpha
        return val

> Define a function to take an (alpha, n, beta) tuple, estimate 10,000 times, and then average to get the expected bias.

In [11]:
from numpy.random import normal as rand
import math

@jit
def calcbias(alpha, n, beta):
    biases = np.zeros(10000)
    for i in range(10000):
        biases[i] = bias(alpha,n,beta)
    avg = np.mean(biases)
    return avg

> Define a function to call the above on the right sets of params, and return the biases.

In [12]:
@jit
def main():
    results = np.zeros(50)
    for i in range(5):
        for j in range(10):
            results[10*i+j] = calcbias(alphas[i],ns[j],1)
    print(results)
    return results

> Now, run computations and time.

In [13]:
%time x = main()

[-0.00771547 -0.0051092  -0.00365801 -0.00336739 -0.00271695 -0.00273799
 -0.00229746 -0.00203893 -0.00197101 -0.00208153 -0.00833809 -0.0063318
 -0.00429925 -0.00379574 -0.00292781 -0.00269557 -0.00303331 -0.00249969
 -0.00243595 -0.00207987 -0.01097336 -0.00656129 -0.00487487 -0.00382249
 -0.0036508  -0.00309616 -0.00287937 -0.00243803 -0.00266576 -0.00210318
 -0.01388954 -0.00739954 -0.0057068  -0.00443808 -0.00399948 -0.00365017
 -0.00321158 -0.00330154 -0.00242881 -0.00257117 -0.05318519 -0.01072521
 -0.00709341 -0.00525823 -0.00452473 -0.00399    -0.00377148 -0.00331809
 -0.00297456 -0.00294658]
CPU times: user 11 s, sys: 13.9 ms, total: 11 s
Wall time: 11 s


> Split the data by alpha.

In [ ]:
from matplotlib import pyplot as plt
vec1 = x[0:10]
vec2 = x[10:20]
vec3 = x[20:30]
vec4 = x[30:40]
vec5 = x[40:50]

> Plot each of the vectors against 'n'

In [ ]:
plt.plot(ns, vec1, 'r-', ns, vec2, 'b-', ns, vec3, 'm-',ns,vec4,'g-',ns,vec5,'k-')
plt.title('Biases vs. Sample Sizes in OLS Estimate of Lag Coefficient')
plt.xlabel('Sample Size')
plt.ylabel('Bias (Absolute Difference)')
plt.show()

![title](figure_1.png)

## What We See

There are a few properties of the OLS estimator which we can extract from the figure:

**Consistency**: There is a clear asymptotic trend to 0 bias. This is because the OLS estimator is a consistent one, which means that it has exactly this property.

**Downward Bias**: All the data points are negative, which confirms our supposition that the OLS estimator is downward-biased.

**Proportionality**: This isn't a formal statistical term, but roughly speaking, we see that the magnitude of the bias increases with the absolute value of alpha.

**Convergence Rate**: We see a logarithmic-type convergence rate, which implies diminishing returns to additional data points, and a clear steep portion to avoid when running simulation.